# ***Distribución RECÁMARAS***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    #if nombre.startswith("df_") and "alfa_q" in nombre and "puebla" in nombre

    if nombre.startswith("df_") and "puebla" in nombre    
    and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_q_puebla',
 'df_feb_2024_puebla',
 'df_jul_2023_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla',
 'df_sep_2023_puebla']

In [4]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Recamaras':'recamaras'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [5]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','recamaras']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

# Total de recamaras en segmentos

In [6]:
def procesar_dataframes(dataframes_list, nombres_df_filtrados):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        bins = [1, 2, 3, 4, 5, 6, float('inf')]
        labels = ['1', '2', '3', '4', '5', '6']
        df['recamaras'] = pd.cut(df['recamaras'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'recamaras']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {nombres_df_filtrados[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, nombres_df_filtrados)

Nombre del DataFrame: df_alfa_q_puebla
recamaras  1   2   3   4  5  6
segmento                      
A          1  28  12   0  2  0
B          9  15  13   0  0  0
C          0  18  11   0  0  0
D          0   4   3   0  0  0
S          0  16  32  13  2  3
Nombre del DataFrame: df_feb_2024_puebla
recamaras   1    2    3    4   5   6
segmento                            
A          12  126  326   64   6  14
B           6  152  363   70  15   9
C          21  206  294   63   5   3
D          23  245  296   50   8   3
E           5  212  441  295  21  10
L           0    8   80   51  14  18
S          12  112  482  213  62  38
Nombre del DataFrame: df_jul_2023_puebla
recamaras   1    2    3    4   5   6
segmento                            
A           8   54  205   33  12  10
B          11  100  211   36  10  11
C           7  112  183   21   4   8
D           5  169  132    8   3   0
E           1   71   34   10   1   7
L           1    5   31   28  11  14
S           6   61  264  161  24 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19672\420837689.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['segmento'] = df['categoria'].str[0]
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19672\420837689.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['categoria'], inplace=True)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_19672\420837689.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

### Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'recamaras' con todos los DataFrame

In [7]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'recamaras'],observed=False).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
recamaras     1       2       3      4      5      6
segmento                                            
A          53.0   607.0  1347.0  272.0   44.0   56.0
B          67.0   684.0  1453.0  275.0   56.0   49.0
C          94.0   877.0  1238.0  204.0   28.0   24.0
D          83.0  1001.0  1115.0  153.0   30.0    6.0
E          16.0   798.0  1280.0  805.0   62.0   44.0
L           1.0    27.0   278.0  210.0   60.0   71.0
S          49.0   461.0  1999.0  967.0  237.0  177.0


In [8]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['recamaras', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_recamaras', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
        yaxis=dict(
            title="Número de Recamaras",
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10), # Ajusta los márgenes (left, right, top, bottom)
    )

    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento      A     B     C     D       E      L     S
recamaras                                             
1            53    67    94    83    16.0    1.0    49
2           607   684   877  1001   798.0   27.0   461
3          1347  1453  1238  1115  1280.0  278.0  1999
4           272   275   204   153   805.0  210.0   967
5            44    56    28    30    62.0   60.0   237
6            56    49    24     6    44.0   71.0   177


In [9]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['recamaras', 'segmento'],observed=False).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_recamaras', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    segmentos = tabla.index
    valores = tabla.iloc[:, 0].tolist()  # Tomamos los valores de la primera columna de la tabla
    etiquetas = tabla.columns[0]  # Tomamos el nombre de la primera columna como etiqueta

    fig1 = go.Figure(data=[go.Pie(
        labels=segmentos, 
        values=valores,
        marker=dict(colors=colores),
        textinfo='label+percent',
        insidetextorientation='radial'        
        )])

    fig1.update_layout(
        # title="Número de Recamaras"
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10), # Ajusta los márgenes (left, right, top, bottom)
        )

    fig1.show()
    return fig1

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig1 = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento      A     B     C     D       E      L     S
recamaras                                             
1            53    67    94    83    16.0    1.0    49
2           607   684   877  1001   798.0   27.0   461
3          1347  1453  1238  1115  1280.0  278.0  1999
4           272   275   204   153   805.0  210.0   967
5            44    56    28    30    62.0   60.0   237
6            56    49    24     6    44.0   71.0   177
